In [1]:
"""
Sequence Number : 5
Ravina Lad
Date : 11th March 2022
CS 5330 Computer Vision
Spring 2021

This Python file includes

- Task Extension 2 : Used LSTM on top of CNN mdoel and this combined module used to train and test MNISTdata
"""
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable
import numpy as np
import sys
from __future__ import print_function
import matplotlib.pyplot as plt

class Args:
    def __init__(self):
        self.cuda = True
        self.no_cuda = False
        self.seed = 1
        self.batch_size = 64
        self.test_batch_size = 1000
        self.epochs = 5
        self.lr = 0.01
        self.momentum = 0.5
        self.log_interval = 10

class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        #x = F.relu(self.fc1(x))
        #x = F.dropout(x, training=self.training)
        #x = self.fc2(x)
        #return F.log_softmax(x, dim=1)
        return x


class Combine(nn.Module):
    def __init__(self):
        super(Combine, self).__init__()
        self.cnn = CNN()
        self.rnn = nn.LSTM(
            input_size=320, 
            hidden_size=64, 
            num_layers=1,
            batch_first=True)
        self.linear = nn.Linear(64,10)

    def forward(self, x):
        batch_size, timesteps, C, H, W = x.size()
        c_in = x.view(batch_size * timesteps, C, H, W)
        c_out = self.cnn(c_in)
        r_in = c_out.view(batch_size, timesteps, -1)
        r_out, (h_n, h_c) = self.rnn(r_in)
        r_out2 = self.linear(r_out[:, -1, :])
        
        return F.log_softmax(r_out2, dim=1)

def train(epoch, model, train_loader, args, optimizer):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        
        data = np.expand_dims(data, axis=1)
        data = torch.FloatTensor(data)
        if args.cuda:
            data, target = data.cuda(), target.cuda()

        data, target = Variable(data), Variable(target)
        optimizer.zero_grad()
        output = model(data)
        
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % args.log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))


def test(model, test_loader, args):
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        
        data = np.expand_dims(data, axis=1)
        data = torch.FloatTensor(data)
        print(target.size)
        
        if args.cuda:
            data, target = data.cuda(), target.cuda()
        data, target = Variable(data, volatile=True), Variable(target)
        output = model(data)
        test_loss += F.nll_loss(
            output, target, size_average=False).item()  # sum up batch loss
        pred = output.data.max(
            1, keepdim=True)[1]  # get the index of the max log-probability
        correct += pred.eq(target.data.view_as(pred)).long().cpu().sum()

    test_loss /= len(test_loader.dataset)
    print(
        '\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
            test_loss, correct, len(test_loader.dataset),
            100. * correct / len(test_loader.dataset)))

# main function (yes, it needs a comment too)
def main(argv):
    
    args = Args()

    args.cuda = not args.no_cuda and torch.cuda.is_available()

    torch.manual_seed(args.seed)
    if args.cuda:
        torch.cuda.manual_seed(args.seed)

    #kwargs = {'num_workers': 1, 'pin_memory': True} if args.cuda else {}

    train_loader = torch.utils.data.DataLoader(
        datasets.MNIST(
            'mnist',
            train=True,
            download=True,
            transform=transforms.Compose([
                transforms.ToTensor(),
                transforms.Normalize((0.1307, ), (0.3081, ))
            ])),
        batch_size=args.batch_size,shuffle=True)

    test_loader = torch.utils.data.DataLoader(
        datasets.MNIST(
            'mnist',
            train=False,
            transform=transforms.Compose([
                transforms.ToTensor(),
                transforms.Normalize((0.1307, ), (0.3081, ))
            ])),
        batch_size=args.test_batch_size,shuffle=True)

    model = Combine()

    optimizer = optim.SGD(model.parameters(), lr=args.lr, momentum=args.momentum)
    for epoch in range(1, args.epochs + 1):
        train(epoch, model, train_loader, args, optimizer)
        test(model, test_loader, args)

    return

if __name__ == "__main__":
    main(sys.argv)



Train Epoch: 1 [0/60000 (0%)]	Loss: 2.282794
Train Epoch: 1 [640/60000 (1%)]	Loss: 2.297014
Train Epoch: 1 [1280/60000 (2%)]	Loss: 2.279903
Train Epoch: 1 [1920/60000 (3%)]	Loss: 2.277450
Train Epoch: 1 [2560/60000 (4%)]	Loss: 2.274046
Train Epoch: 1 [3200/60000 (5%)]	Loss: 2.276139
Train Epoch: 1 [3840/60000 (6%)]	Loss: 2.269833
Train Epoch: 1 [4480/60000 (7%)]	Loss: 2.258232
Train Epoch: 1 [5120/60000 (9%)]	Loss: 2.252633
Train Epoch: 1 [5760/60000 (10%)]	Loss: 2.248990
Train Epoch: 1 [6400/60000 (11%)]	Loss: 2.242020
Train Epoch: 1 [7040/60000 (12%)]	Loss: 2.246799
Train Epoch: 1 [7680/60000 (13%)]	Loss: 2.210749
Train Epoch: 1 [8320/60000 (14%)]	Loss: 2.209940
Train Epoch: 1 [8960/60000 (15%)]	Loss: 2.190331
Train Epoch: 1 [9600/60000 (16%)]	Loss: 2.183873
Train Epoch: 1 [10240/60000 (17%)]	Loss: 2.124814
Train Epoch: 1 [10880/60000 (18%)]	Loss: 2.156274
Train Epoch: 1 [11520/60000 (19%)]	Loss: 2.115807
Train Epoch: 1 [12160/60000 (20%)]	Loss: 2.112792
Train Epoch: 1 [12800/60000 (

/tmp/ipykernel_105531/3158038649.py:110: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  data, target = Variable(data, volatile=True), Variable(target)
/home/ravina/.local/lib/python3.8/site-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


<built-in method size of Tensor object at 0x7f2d5806cf90>
<built-in method size of Tensor object at 0x7f2d575b3810>
<built-in method size of Tensor object at 0x7f2d575b32c0>
<built-in method size of Tensor object at 0x7f2d575b1360>
<built-in method size of Tensor object at 0x7f2d58388c70>
<built-in method size of Tensor object at 0x7f2d575b3d60>
<built-in method size of Tensor object at 0x7f2d575b1cc0>
<built-in method size of Tensor object at 0x7f2d57fc0400>
<built-in method size of Tensor object at 0x7f2d5806cf90>

Test set: Average loss: 0.4671, Accuracy: 8837/10000 (88%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.706460
Train Epoch: 2 [640/60000 (1%)]	Loss: 0.574211
Train Epoch: 2 [1280/60000 (2%)]	Loss: 0.643939
Train Epoch: 2 [1920/60000 (3%)]	Loss: 0.564583
Train Epoch: 2 [2560/60000 (4%)]	Loss: 0.544560
Train Epoch: 2 [3200/60000 (5%)]	Loss: 0.614019
Train Epoch: 2 [3840/60000 (6%)]	Loss: 0.626696
Train Epoch: 2 [4480/60000 (7%)]	Loss: 0.540440
Train Epoch: 2 [5120/60000 (9%)]	Loss

Train Epoch: 3 [30080/60000 (50%)]	Loss: 0.282677
Train Epoch: 3 [30720/60000 (51%)]	Loss: 0.156353
Train Epoch: 3 [31360/60000 (52%)]	Loss: 0.251841
Train Epoch: 3 [32000/60000 (53%)]	Loss: 0.156513
Train Epoch: 3 [32640/60000 (54%)]	Loss: 0.320144
Train Epoch: 3 [33280/60000 (55%)]	Loss: 0.224865
Train Epoch: 3 [33920/60000 (57%)]	Loss: 0.206936
Train Epoch: 3 [34560/60000 (58%)]	Loss: 0.240691
Train Epoch: 3 [35200/60000 (59%)]	Loss: 0.225577
Train Epoch: 3 [35840/60000 (60%)]	Loss: 0.156334
Train Epoch: 3 [36480/60000 (61%)]	Loss: 0.266990
Train Epoch: 3 [37120/60000 (62%)]	Loss: 0.128492
Train Epoch: 3 [37760/60000 (63%)]	Loss: 0.220319
Train Epoch: 3 [38400/60000 (64%)]	Loss: 0.211097
Train Epoch: 3 [39040/60000 (65%)]	Loss: 0.227383
Train Epoch: 3 [39680/60000 (66%)]	Loss: 0.426133
Train Epoch: 3 [40320/60000 (67%)]	Loss: 0.403951
Train Epoch: 3 [40960/60000 (68%)]	Loss: 0.303162
Train Epoch: 3 [41600/60000 (69%)]	Loss: 0.252395
Train Epoch: 3 [42240/60000 (70%)]	Loss: 0.207342


Train Epoch: 5 [640/60000 (1%)]	Loss: 0.202828
Train Epoch: 5 [1280/60000 (2%)]	Loss: 0.195498
Train Epoch: 5 [1920/60000 (3%)]	Loss: 0.110004
Train Epoch: 5 [2560/60000 (4%)]	Loss: 0.187101
Train Epoch: 5 [3200/60000 (5%)]	Loss: 0.139772
Train Epoch: 5 [3840/60000 (6%)]	Loss: 0.180480
Train Epoch: 5 [4480/60000 (7%)]	Loss: 0.254623
Train Epoch: 5 [5120/60000 (9%)]	Loss: 0.262520
Train Epoch: 5 [5760/60000 (10%)]	Loss: 0.218398
Train Epoch: 5 [6400/60000 (11%)]	Loss: 0.167351
Train Epoch: 5 [7040/60000 (12%)]	Loss: 0.137745
Train Epoch: 5 [7680/60000 (13%)]	Loss: 0.267980
Train Epoch: 5 [8320/60000 (14%)]	Loss: 0.162586
Train Epoch: 5 [8960/60000 (15%)]	Loss: 0.119056
Train Epoch: 5 [9600/60000 (16%)]	Loss: 0.204976
Train Epoch: 5 [10240/60000 (17%)]	Loss: 0.202319
Train Epoch: 5 [10880/60000 (18%)]	Loss: 0.175167
Train Epoch: 5 [11520/60000 (19%)]	Loss: 0.104823
Train Epoch: 5 [12160/60000 (20%)]	Loss: 0.157889
Train Epoch: 5 [12800/60000 (21%)]	Loss: 0.218087
Train Epoch: 5 [13440/60